# 06 - Kaggle - bike share system - Ordinary Linear Regression


For problem formulation refer to **"01 - Kaggle - bike share system - problem formulation.ipynb"**.
In section **"02 - Kaggle - bike share system - Data preprocessing.ipynb"** we transformed the raw data and extracted time, date, and dummy matrices. The results are stored in two formats:
 * In `train_prep_orig.csv` and `test_prep_orig.csv` the categorical data are in the original form.
 * In `train_prep_dum.csv` and `test_prep_dum.csv` the categorical data are converted to dummy matrices. 

In section **03 - Kaggle - bike share system - data visualization.ipynb** we ploted the average of customers at different time periods over 2011 and 2012 and discused the pattern of customer behavior. We concluded by a decision to consider these average values as new features of the problem so that the machine learning model will be able to use them as the basis values and the other features apply the necessary correction to make use closer to the actual values. In **04 - Kaggle - bike share system - Adding customer average to the features.ipynb** we added the average customers as new featres to the data sets. We finally, update the data sets as:
 * In `train_prep_orig_avg.csv` and `test_prep_orig_avg.csv` the categorical data are in the original form.
 * In `train_prep_dum_avg.csv` and `test_prep_dum_avg.csv` the categorical data are converted to dummy matrices. 
 
 
In section **5 - Kaggle - bike share system - data analysis.ipynb** we visulaized and analyzed the data. We saw that instead of using average of customers as features, it is better if we use the target to be the deviation of customer from the average value.  


In this section we train our first machine learning model, which is linear regression. ordinary We use the data in `train_prep_orig_avg.csv`, which includes:
<center> temp|  atemp | humidity | windspeed | year | season | month | weekday | hour | workingday | holiday | weather | avg_casual | avg_registered | avg_tot | casual | registered | count
 </center>

- **temp**: temperature in Celsius. 

- **atemp**: "feels like" temperature in Celsius.

- **humidity**: relative humidity

- **windspeed**: wind speed

- **year**: 2011 or 2012

- **season**: Kaggle's [website](https://www.kaggle.com/c/bike-sharing-demand/data) says "`1 = spring, 2 = summer, 3 = fall, 4 = winter`", but the season indecies in the dataset correspond to 
    - 1 = Winter (January-March)
    - 2 = Spring (April-June)
    - 3 = Summer (July-September)
    - 4 = Fall (October-December)
    
- **month**: The month as January=1, December=12  

- **weekday**: The day of the week with Monday=0, Sunday=6

- **hour**: The hours of the datetime 0 - 23

- **workingday**: whether the day is neither a weekend nor holiday
     - 0 = day is weekend or holiday
     - 1 = otherwise 

- **holiday**: whether the day is considered a holiday 
    - 0 = non-holiday
    - 1 = holiday

- **weather**: encoded to make explicit various extreme weather events
    - 1 = Clear, Few clouds, Partly cloudy, Partly cloudy 
    - 2 = Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3 = Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds)
    - 4 = Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

- **casual**: number of non-registered user rentals initiated

- **registered**: number of registered user rentals initiated

- **tot**: number of total rentals (casual + registered)

- **avg_casual**: the average number of casual customers per weekday per month per year in a time period
- **avg_registered**:   the average number of registered customers per weekday per month per year in a time period
- **avg_tot**:  the average number of total customers per weekday per month per year in a time period





### Basic settings and importing the libraries

In [8]:
# Resets the namespace by removing all names defined by the user without asking for confirmation
%reset -f


# Panas is used for DataFrame
import pandas as pd

# NumPy is used for manipulating arrays
import numpy as np

# MatPlotLib is used for plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# the output of plotting commands is displayed inline directly below the code cell that produced it.
%matplotlib inline

# Seaborn is used for statistical plotting
import seaborn as sns

# Used for display dataframes as html tables
from IPython.display import display

The Root Mean Squared Logarithmic Error (RMSLE) is calculated as

$$\epsilon = \sqrt{{1\over n} \sum_{i=1}^n \left[ \log(p_i+1) - \log(a_i+1)\right]^2}$$

*Where:*

- $n_i$ is the number of hours in the test set
- $p_i$ is your predicted count
- $a_i$ is the actual count
- $\log(x)$ is the natural logarithm



In [69]:
def rmsle (y, y_pred):
    return np.sqrt(((np.log1p(y_pred) - np.log1p(y))**2).mean())

### Importing the train data from `train_prep_orig.csv`

In [13]:
#Load train data
data_train = pd.read_csv('data/train_prep_orig_avg.csv')

print "The shape of the train dataset:", data_train.shape
display(data_train.head())


The shape of the train dataset: (10886, 18)


,temp,atemp,humidity,windspeed,year,season,month,weekday,hour,workingday,holiday,weather,avg_casual,avg_registered,avg_tot,casual,registered,tot
0,9.84,14.395,81,0,2011,1,1,5,0,0,0,1,3,21.166667,24.166667,3,13,16
1,9.02,13.635,80,0,2011,1,1,5,1,0,0,1,3,21.166667,24.166667,8,32,40
2,9.02,13.635,80,0,2011,1,1,5,2,0,0,1,1,7.500000,8.500000,5,27,32
3,9.84,14.395,75,0,2011,1,1,5,3,0,0,1,1,7.500000,8.500000,3,10,13
4,9.84,14.395,75,0,2011,1,1,5,4,0,0,1,1,7.500000,8.500000,0,1,1


## Features and targets

In [14]:
cat_var = ['year','season', 'month', 'weekday', 'hour', 'workingday', 'holiday', 'weather']
num_var = ['temp', 'atemp', 'humidity', 'windspeed']
avg_var = ['avg_casual','avg_registered','avg_tot']
dev_var = ['dev_casual','dev_registered','dev_tot']
target_var = ['casual', 'registered', 'tot']


## Deviations of customers from average

In [15]:
data_train['dev_casual'] = data_train['casual'] - data_train['avg_casual'] 
data_train['dev_registered'] = data_train['registered'] - data_train['avg_registered'] 
data_train['dev_tot'] = data_train['tot'] - data_train['avg_tot'] 

data_train.head()

,temp,atemp,humidity,windspeed,year,season,month,weekday,hour,workingday,...,weather,avg_casual,avg_registered,avg_tot,casual,registered,tot,dev_casual,dev_registered,dev_tot
0,9.84,14.395,81,0,2011,1,1,5,0,0,...,1,3,21.166667,24.166667,3,13,16,0,-8.166667,-8.166667
1,9.02,13.635,80,0,2011,1,1,5,1,0,...,1,3,21.166667,24.166667,8,32,40,5,10.833333,15.833333
2,9.02,13.635,80,0,2011,1,1,5,2,0,...,1,1,7.500000,8.500000,5,27,32,4,19.500000,23.500000
3,9.84,14.395,75,0,2011,1,1,5,3,0,...,1,1,7.500000,8.500000,3,10,13,2,2.500000,4.500000
4,9.84,14.395,75,0,2011,1,1,5,4,0,...,1,1,7.500000,8.500000,0,1,1,-1,-6.500000,-7.500000


### Selecting feature ad target variables

In [34]:
X = data_train.loc[:,'temp':'weather']
print "features"
display(X.head())

Y_avg = data_train.loc[:,'avg_casual':'avg_tot']
print "target (average)"
display(Y_avg.head())

Y_org = data_train.loc[:,'casual':'tot']
print "target"
display(Y_org.head())

Y_dev = data_train.loc[:,'dev_casual':'dev_tot']
print "target (deviation)"
display(Y_dev.head())

features


,temp,atemp,humidity,windspeed,year,season,month,weekday,hour,workingday,holiday,weather
0,9.84,14.395,81,0,2011,1,1,5,0,0,0,1
1,9.02,13.635,80,0,2011,1,1,5,1,0,0,1
2,9.02,13.635,80,0,2011,1,1,5,2,0,0,1
3,9.84,14.395,75,0,2011,1,1,5,3,0,0,1
4,9.84,14.395,75,0,2011,1,1,5,4,0,0,1


target (average)


,avg_casual,avg_registered,avg_tot
0,3,21.166667,24.166667
1,3,21.166667,24.166667
2,1,7.500000,8.500000
3,1,7.500000,8.500000
4,1,7.500000,8.500000


target


,casual,registered,tot
0,3,13,16
1,8,32,40
2,5,27,32
3,3,10,13
4,0,1,1


target (deviation)


,dev_casual,dev_registered,dev_tot
0,0,-8.166667,-8.166667
1,5,10.833333,15.833333
2,4,19.500000,23.500000
3,2,2.500000,4.500000
4,-1,-6.500000,-7.500000


## Ordinary Linear Regression

Here we fit an ordinary least square regression model to the data and calculate the $R^2$ score.

The coefficient $R^2$ is defined as $(1 - u/v)$, where $u$ is the regression sum of squares $\sum_i (y_{true,i} - y_{pred,i})^2$ and $v$ is the residual sum of squares $\sum_i (y_{true,i} - \bar{y}_{true})^2$. Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of $y$, disregarding the input features, would get a $R^2$ score of 0.0.

### Split to train and test samples

In [37]:
from sklearn.cross_validation import train_test_split

X_train, X_test, Y_avg_train, Y_avg_test, Y_org_train, Y_org_test, Y_dev_train, Y_dev_test = train_test_split(
    X, 
    Y_avg,
    Y_org, 
    Y_dev, 
    test_size = 0.30, 
    random_state = 42)

### Linea Regression

Here we only focus on total number of customers. We may extend the analysis later to casual and registered customers.

In [82]:
from sklearn.linear_model import LinearRegression

def fit_olr (X_train, Y_train, X_test, Y_test):
    lr = LinearRegression()
    lr.fit(X_train,Y_train)    
    print "intercept: ",  lr.intercept_ 
    print ("Train R^2: %.3f " % lr.score(X_train,Y_train))
    print ("Test R^2: %.3f " % lr.score(X_test,Y_test))
    print ("Train rmsle: %.3f " % rmsle(Y_train, lr.predict(X_train)))
    print ("Test rmsle: %.3f " % rmsle(Y_test, lr.predict(X_test)))
     

def fit_olr_dev (X_train, Y_train, Y_0_train, X_test, Y_test, Y_0_test):
    lr = LinearRegression()
    lr.fit(X_train,Y_train)    
    print "intercept: ",  lr.intercept_ 
    print ("Train R^2: %.3f " % lr.score(X_train,Y_train))
    print ("Test R^2: %.3f " % lr.score(X_test,Y_test))
    print ("Train rmsle: %.3f " % rmsle(Y_train + Y_0_train, lr.predict(X_train) + Y_0_train))
    print ("Test rmsle: %.3f " % rmsle(Y_test + Y_0_test, lr.predict(X_test) + Y_0_test))
    
print "\nFitting the original values (all features)"     
fit_olr(X_train, Y_org_train.tot, X_test ,Y_org_test.tot)

print "\nFitting the deviation values (all features)" 
fit_olr_dev(X_train, Y_dev_train.dev_tot, Y_avg_train.avg_tot, X_test ,Y_dev_test.dev_tot, Y_avg_test.avg_tot )


Fitting the original values (all features)
intercept:  -163745.665934
Train R^2: 0.390 
Test R^2: 0.387 
Train rmsle: 1.181 
Test rmsle: 1.167 

Fitting the deviation values (all features)
intercept:  9266.63557826
Train R^2: 0.039 
Test R^2: 0.051 
Train rmsle: 0.718 
Test rmsle: 0.711 


We see that fitting the deviation values leads to smaller RMSLE. In section **5 - Kaggle - bike share system - data analysis.ipynb** we saw that the deviation vales have very low (almost zero) correlation with `year`, `season`, `month`, `hour`, and `weekday`. So, let's take these out of the data and do the analysis again:

In [88]:
display(X_train.head())

correlated_features = ['temp','atemp','humidity','windspeed','weather','workingday','holiday']

print "\nFitting the deviation values (all features)" 
fit_olr_dev(X_train[correlated_features]
            , Y_dev_train.dev_tot, Y_avg_train.avg_tot, 
            X_test [correlated_features]
            ,Y_dev_test.dev_tot, Y_avg_test.avg_tot )

,temp,atemp,humidity,windspeed,year,season,month,weekday,hour,workingday,holiday,weather
613,9.02,9.090,32,39.0007,2011,1,2,1,17,1,0,1
4030,22.14,25.760,68,12.9980,2011,3,9,6,23,0,0,1
3582,26.24,28.790,83,0.0000,2011,3,8,4,4,1,0,1
10101,9.02,11.365,69,8.9981,2012,4,11,1,6,1,0,1
1430,13.12,14.395,81,30.0026,2011,2,4,1,11,1,0,3



Fitting the deviation values (all features)
intercept:  36.9081915329
Train R^2: 0.034 
Test R^2: 0.044 
Train rmsle: 0.710 
Test rmsle: 0.699 


We have some small improvements, but it is not significant and may be due to the train-test split. Ordinary linear regression is not a good choice for this problem since the system is nonlinear.